In [2]:
!pip install gdown
import gdown
# Replace 'FILE_ID' with the actual ID from your Drive share link
file_id = '1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj' 
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'augmented_pepper_dataset.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj
From (redirected): https://drive.google.com/uc?id=1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj&confirm=t&uuid=fe720751-5b3e-46f3-b86a-516916905e7c
To: /kaggle/working/augmented_pepper_dataset.zip
100%|██████████| 4.00G/4.00G [00:29<00:00, 135MB/s] 


'augmented_pepper_dataset.zip'

In [3]:
import os
import zipfile
import shutil

zip_path = 'augmented_pepper_dataset.zip'
extract_to = '/kaggle/working/training_data'

# 1. Force a clean start by removing the old folder if it exists
if os.path.exists(extract_to):
    shutil.rmtree(extract_to)
    print(f"🗑️ Cleaned up existing folder: {extract_to}")

# 2. Extract the data
print("Extracting augmented dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print("✅ Extraction complete.")

Extracting augmented dataset...
✅ Extraction complete.


In [4]:
import os

# This will list everything in your working directory
for root, dirs, files in os.walk('/kaggle/working/'):
    level = root.replace('/kaggle/working/', '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")

/
.virtual_documents/
training_data/
    content/
        dataset/
            HEALTHY/
                black pepper RAW/
                black_pepper_healthy/
                augmented/
                Black pepper detection1.v1i.yolov8/
                    test/
                        images/
                        labels/
                    train/
                        images/
                        labels/
                    valid/
                        images/
                        labels/
                MERGED/
            UNHEALTHY/
                Pollu_Disease/
                black_pepper_yellow_mottle_virus/
                black_pepper_leaf_blight/
                Slow-Decline/
                augmented/
                Footrot/
                MERGED/


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset, Dataset, ConcatDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import glob
from PIL import Image
# Added for speed and memory efficiency
from torch.cuda.amp import GradScaler, autocast 

# 1. Path Configuration
healthy_aug_dir = '/kaggle/working/training_data/content/dataset/HEALTHY/augmented'
unhealthy_aug_dir = '/kaggle/working/training_data/content/dataset/UNHEALTHY/augmented'

# 2. Data Transformations
data_transforms = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 3. Custom Loader
class AugmentedPepperDataset(Dataset):
    def __init__(self, root_dir, transform=None, label=0):
        self.image_paths = glob.glob(os.path.join(root_dir, '*.*'))
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

dataset_healthy = AugmentedPepperDataset(healthy_aug_dir, transform=data_transforms, label=0)
dataset_unhealthy = AugmentedPepperDataset(unhealthy_aug_dir, transform=data_transforms, label=1)
full_dataset = ConcatDataset([dataset_healthy, dataset_unhealthy])

print(f"Detected classes: ['HEALTHY', 'UNHEALTHY']")
print(f"Total augmented images found: {len(full_dataset)}")

# 4. Train/Test Split
indices = list(range(len(full_dataset)))
targets = [0] * len(dataset_healthy) + [1] * len(dataset_unhealthy)

train_idx, test_idx = train_test_split(
    indices,
    train_size=0.8,
    random_state=42,
    stratify=targets
)

train_data = Subset(full_dataset, train_idx)
test_data = Subset(full_dataset, test_idx)

# SPEED BOOST: Added num_workers and pin_memory
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# 5. Initialize ResNet18
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# STABILITY: Initialize Scaler for Mixed Precision
scaler = GradScaler()

# 6. Training Loop
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            
            # SPEED BOOST: Use autocast for FP16 training
            with autocast():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            # STABILITY: Scaled backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            pbar.set_postfix(loss=loss.item())

        epoch_acc = running_corrects.double() / len(train_data)
        print(f"Training Accuracy: {epoch_acc:.4f}")
        
        # RESUME SUPPORT: Save a checkpoint after every epoch
        checkpoint_name = f'pepper_resnet18_epoch_{epoch+1}.pth'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'acc': epoch_acc
        }, checkpoint_name)
        print(f"Saved checkpoint: {checkpoint_name}")
        
        torch.cuda.empty_cache() 

    return model

# Start training
model = train_model(model, criterion, optimizer, num_epochs=10)

# 7. Save Final Model
torch.save(model.state_dict(), 'pepper_resnet18_augmented_500px.pth')
print("✅ Model weights saved! Download it from the 'Output' pane.")

Detected classes: ['HEALTHY', 'UNHEALTHY']
Total augmented images found: 47952
Training on: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]
/tmp/ipykernel_55/373380478.py:79: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/2398 [00:00<?, ?it/s]/tmp/ipykernel_55/373380478.py:95: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10: 100%|██████████| 2398/2398 [04:46<00:00,  8.36it/s, loss=0.021]   


Training Accuracy: 0.9910
Saved checkpoint: pepper_resnet18_epoch_1.pth


Epoch 2/10: 100%|██████████| 2398/2398 [04:45<00:00,  8.41it/s, loss=3.82e-5] 


Training Accuracy: 0.9963
Saved checkpoint: pepper_resnet18_epoch_2.pth


...

Training Accuracy: 0.9990
Saved checkpoint: pepper_resnet18_epoch_8.pth


Epoch 9/10: 100%|██████████| 2398/2398 [04:44<00:00,  8.42it/s, loss=3.14e-5] 


Training Accuracy: 0.9993
Saved checkpoint: pepper_resnet18_epoch_9.pth


Epoch 10/10: 100%|██████████| 2398/2398 [04:44<00:00,  8.43it/s, loss=1.19e-7] 


Training Accuracy: 0.9993
Saved checkpoint: pepper_resnet18_epoch_10.pth
✅ Model weights saved! Download it from the 'Output' pane.
